# Experiment Notebook
Notebook to inspect the results of a single experiment

In [ ]:
import pandas as pd
import graphviz
import typing

from collections import defaultdict


from lib.metrics2 import *
from lib.message import *
from lib.analysis import KadPubSubAnalyzer
from lib.experiment import PubSubExperiment, PubSubExperimentResults


In [ ]:
experiment_results = PubSubExperimentResults.load_from_file(
    "experiments/experiment_3_v2_flood.json"
)
exp = KadPubSubAnalyzer.from_experiment_results(experiment_results)
exp.experiment


In [ ]:
exp.messages()


In [ ]:
receives = exp.metrics_with(lambda m: isinstance(m, PubSubMessageReceived))
messages = exp.messages()
message_receive_counts = {
    m: len([r for r in receives if r.message_id == m]) for m in messages
}
message_receive_counts = {
    item for item in message_receive_counts.items() if item[1] >= 1
}
message_receive_counts


In [ ]:
msgid = list(exp.messages())[0]  # "947381e5-d811-4e11-9e39-f515a0132aa3"
topic = exp.topic_of_message(msgid)
subscribers = exp.topic_subscribers(topic)
broadcasts = exp.metrics_with(
    lambda m: isinstance(m, MessageSent)
    and isinstance(m.message, BroadcastMessage)
    and m.message.topic == topic
    and m.message.message_id == msgid
)
haves = exp.metrics_with(
    lambda m: isinstance(m, MessageSent)
    and isinstance(m.message, BroadcastHave)
    and m.message.topic == topic
    and m.message.message_id == msgid
)
wanthaves = exp.metrics_with(
    lambda m: isinstance(m, MessageSent)
    and isinstance(m.message, BroadcastWant)
    and m.message.topic == topic
    and m.message.message_id == msgid
)
sends = exp.metrics_with(
    lambda m: isinstance(m, PubSubMessageSent) and m.message_id == msgid
)

dot = graphviz.Digraph("Message Broadcast Tree")
dot.node(broadcasts[0].node)
for sub in subscribers:
    dot.node(sub)
for bcast in broadcasts:
    dot.edge(bcast.node, bcast.destination, color="green")
for have in haves:
    dot.edge(have.node, have.destination, color="blue")
for wanthave in wanthaves:
    dot.edge(wanthave.node, wanthave.destination, color="red")

print(
    f"First sender of message {msgid} was {broadcasts[0].node} with subscription {broadcasts[0].node in subscribers}"
)
print(f"Topic has {len(subscribers)} subscribers")
dot


In [ ]:
# rt = exp.routing_table_snapshot("5000", topic, exp.timestamp_of_message(msgid))
# print(f"Routing table node: {rt.node}")
# print(f"Routing table topic: {rt.topic}")
# print("Buckets:")
# for bucket in rt.buckets:
#     print(f"  {bucket}")


In [ ]:
delivers = exp.metrics_with(
    lambda m: isinstance(m, PubSubMessageReceived)
    and m.message_id == msgid
    and m.delivered
)
print(f"Message {msgid} was delivered to {len(delivers)} nodes")
print(f"Message was initially sent by {broadcasts[0].node}")
print(
    f"Message was not delivered to {subscribers - set(map(lambda m: m.node, delivers))}"
)


In [ ]:
exp.reliability().resample("1s").mean().plot()


In [ ]:
exp.reliability().plot(kind="hist")


In [ ]:
dot = graphviz.Digraph("Identifier Space")
depth = 6
for i in range(depth):
    n = 2**i
    subdot = graphviz.Digraph()
    for j in range(n):
        parent_label = f"{i-1}-{j//2}"
        label = "" if i != depth - 1 else f"{j}"
        subdot.node(f"{i}-{j}", label=label, level=f"{i}")
        if not (i == 0 and j == 0):
            dot.edge(parent_label, f"{i}-{j}")
    subdot.attr(rank="same")
    dot.subgraph(subdot)
dot
